In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
ROOT_DIR = '/content/gdrive/My Drive/Automated_vechile_counter_car'

In [ ]:
!pip install ultralytics

In [ ]:
import os

# from ultralytics import YOLO


# # Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch

# # Use the model
# results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=1)  # train the model


from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=100, imgsz=640)


In [ ]:
from google.colab import files


!zip -r /content/runs.zip /content/runs

files.download('/content/runs.zip')

In [ ]:
import os
import cv2
import torch

from ultralytics import YOLO

# Load the YOLOv8 model (choose 'yolov8n.pt', 'yolov8s.pt', etc. for different sizes)
# ROOT_DIR, "google_colab_config.yaml"

model = YOLO(ROOT_DIR, "last.pt")  # or another version of YOLOv8 (e.g., yolov8s.pt for small)
# model = YOLO('./runs/detect/train/weights/last.pt')  # or another version of YOLOv8 (e.g., yolov8s.pt for small)

# Load the video file
input_video_path = 'video.mp4'
output_video_path = 'out.mp4'

# Open the video using OpenCV
video_capture = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Iterate over each frame
frame_count = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()  # Read a frame
    if not ret:
        break

    # Apply YOLOv8 object detection
    results = model(frame)[0]

    # Iterate through the detections and draw bounding boxes
    for result in results.boxes.data.tolist():  # Each detection in the format [x1, y1, x2, y2, conf, class]
        x1, y1, x2, y2, conf, cls = result[:6]
        label = f'{model.names[cls]} {conf:.2f}'

        # Draw bounding box and label on the frame
        if conf > 0.5:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4)  # Bounding box
        # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the processed frame to the output video
    out_video.write(frame)

    # Print progress
    frame_count += 1
    print(f'Processed frame {frame_count}/{total_frames}')

# Release resources
video_capture.release()
out_video.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')
